In [3]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, TimeoutException, ElementClickInterceptedException
from time import sleep

# Define .csv

In [4]:
import pandas as pd

# Define columns for the DataFrame
columns = [
    'company_name', 
    'recruiter_name',
    'job_title', 
    'location', 
    'employment_type',
    'job_level', 
    'job_function', 
    'salary_lower', 
    'salary_upper', 
    'salary_period',
    'num_applications', 
    'posted_date',
    'address',
    'experience',
    'closing_date',
    'job_description'
]

# Create an empty DataFrame with the defined columns
jobs = pd.DataFrame(columns=columns)

# Functions

In [5]:
# Define dictionaries/lists for categories
locations = ['Central', 'East', 'North', 'South', 'West', 'Islandwide']
employment_types = ['Permanent', 'Full Time', 'Part Time', 'Contract', 'Flexi-work', 'Temporary', 'Freelance', 'Internship/Attachment']
job_levels = ['Senior Management', 'Middle Management', 'Manager', 'Professional', 'Senior Executive', 'Executive', 'Junior Executive', 'Non-executive', 'Fresh/Entry Level']

# Example strings
example_strings = [
    "CentralContractJunior ExecutiveEngineering ...",
    "IslandwideContract ...Fresh/Entry LevelEducation And Training ...",
    "SouthInternship/AttachmentFresh/Entry LevelHealthcare / Pharmaceutical",
    "IslandwideContract ...ProfessionalBanking And Finance ...",
    "Contract ...ProfessionalBanking And Finance ..."
]

# Function to split the string and determine values for each category
def parse_contract_type(string):
    clean_str = string.replace(" ...", "")
    res_location, res_employment_type, res_job_level, res_job_function = None, None, None, None
    for location in locations:
        if location in clean_str:
            res_location = location
            clean_str = clean_str.replace(location, '')
            break
    for employment_type in employment_types:
        if employment_type in clean_str:
            res_employment_type = employment_type
            clean_str = clean_str.replace(employment_type, '')
            break
    for job_level in job_levels:
        if job_level in clean_str:
            res_job_level = job_level
            clean_str = clean_str.replace(job_level, '')
            break
    res_job_function = clean_str.strip()

    return res_location, res_employment_type, res_job_level, res_job_function  

# Print test
for i in example_strings:
    print(list(parse_contract_type(i)))

['Central', 'Contract', 'Executive', 'Junior Engineering']
['Islandwide', 'Contract', 'Fresh/Entry Level', 'Education And Training']
['South', 'Internship/Attachment', 'Fresh/Entry Level', 'Healthcare / Pharmaceutical']
['Islandwide', 'Contract', 'Professional', 'Banking And Finance']
[None, 'Contract', 'Professional', 'Banking And Finance']


In [6]:
def parse_salary_range(salary_range):
    # Extract upper and lower salary values
    salary_values = re.findall(r'(\d+(?:,\d+)?)', salary_range)
    salary_lower = int(salary_values[0].replace(',', '')) if salary_values else None
    salary_upper = int(salary_values[1].replace(',', '')) if len(salary_values) > 1 else None
    
    return salary_lower, salary_upper

In [7]:
import re

def parse_applications(applications):
    if applications:
        # Extract only the integer number
        match = re.search(r'\d+', applications)
        num_applications = int(match.group()) if match else None
    else:
        num_applications = None
    return num_applications

In [8]:
'''
GMP RECRUITMENT SERVICES (S) PTE LTD
Junior Data Scientist
CentralContractJunior ExecutiveEngineering ...
0% skills matched
$3,500to$7,000
Monthly
0 application
Posted yesterday
'''

def store_card_info(card, row_no):
    # Split the card information into lines
    lines = card.split('\n')
    
    # Extract relevant information
    company_name, recruiter_name, job_title, contract_type, salary_range, salary_period, applications, posted_date = (None, None, None, None, None, None, None, None)
    if(len(lines) > 8): 
        '''
        for cases such as
        PETROS-CONSULTING PTE. LTD.RECRUITER
        for ST ENGINEERING IHQ PTE. LTD.
        '''
        recruiter_name = lines[0].replace("RECRUITER", "")
        company_name = lines[1].replace("for ", "")
        job_title = lines[2]
        contract_type = lines[3]
        # skills_match = lines[4]
        salary_range = lines[5]
        salary_period = lines[6]
        applications = None #lines[7]
        posted_date = None #lines[8]
    else:
        company_name = lines[0]
        recruiter_name = None
        job_title = lines[1]
        contract_type = lines[2]
        # skills_match = lines[3]
        salary_range = lines[4]
        salary_period = lines[5]
        applications = None #lines[6]
        posted_date = None #lines[7]
    
    # Parse contract_type
    location, employment_type, job_level, job_function = parse_contract_type(contract_type)
    
    # Parse salary_range
    salary_lower, salary_upper = parse_salary_range(salary_range)
    
    # Parse applications
    num_applications = applications #parse_applications(applications)

    # Initialize additional columns
    address = None
    experience = None
    closing_date = None
    job_description = None
    
    # Write
    jobs.loc[row_no] = [
        company_name, 
        recruiter_name,
        job_title, 
        location, 
        employment_type,
        job_level, 
        job_function, 
        salary_lower, 
        salary_upper, 
        salary_period,
        num_applications, 
        posted_date,
        address,
        experience,
        closing_date,
        job_description
    ]

In [9]:
def parse_posting_info(posting_info):
    # Define the pattern to match "applicationsPosted" and "Closing on" as separators
    pattern = r'( applicationPosted | applicationsPosted |Closing on )'

    applications, filler_1, posted_date, filler_2, closing_date = re.split(pattern, posting_info)

    return applications, posted_date, closing_date

posting_info_eg = [
    "4 applicationsPosted 30 Mar 2024Closing on 29 Apr 2024",
    "30 applicationsPosted 29 Mar 2024Closing on 05 Apr 2024",
    "240 applicationsPosted 29 Mar 2024Closing on 28 Apr 2024",
    "0 applicationPosted 31 Mar 2024Closing on 30 Apr 2024",
    "1 applicationPosted 31 Mar 2024Closing on 30 Apr 2024"
]

# Split each string and print the results
for string in posting_info_eg:
    print(parse_posting_info(string))

('4', '30 Mar 2024', '29 Apr 2024')
('30', '29 Mar 2024', '05 Apr 2024')
('240', '29 Mar 2024', '28 Apr 2024')
('0', '31 Mar 2024', '30 Apr 2024')
('1', '31 Mar 2024', '30 Apr 2024')


## Web Scrape function

In [10]:
from random import uniform
# uniform(0.1, 1)

In [11]:
from datetime import datetime
# datetime.today() > datetime.strptime('01 Jan 2024', '%d %b %Y')

In [48]:
def scrape_mycareersfuture(search_term, furthest_date='01 Jan 2023'):
    '''
    furthest_date format: DD Mmm YYYY
    sleep to avoid kick out as bot
    '''
    driver = webdriver.Chrome()
    wait = WebDriverWait(driver, 1)

    page_no = 0
    driver.get(f"https://www.mycareersfuture.gov.sg/search?search={search_term}&sortBy=new_posting_date&page={page_no}")
    row_no = -1
    proceed_to_next_page = True
    furthest_date = datetime.strptime(furthest_date, '%d %b %Y')
    cur_date = datetime.today()
    while proceed_to_next_page and cur_date > furthest_date:
        # sleep(uniform(1, 5))
        for card_id in range(22):
            # sleep(uniform(1, 2))
            try:
                print(f"ROW NO: {row_no}, CARD NO: {card_id}, PAGE NO: {page_no}")
                
                # url = wait.until(EC.element_to_be_clickable((By.XPATH, f"//div[@id='job-card-{card_id}']/div[@class='card relative']/a[@class='bg-white mb3 w-100 dib v-top pa3 no-underline flex-ns flex-wrap JobCard__card___22xP3']/div[@class='w-80-l w-100 flex flex-wrap']/div[@class='w-100 order-0 pr3-ns pr0']/section[@class='flex']/div[@class='pl3 JobCard__job-title-flex___2R-sW']/div[@class='w-100']/span[@class='f4-5 fw6 mv0 dib mr2 brand-sec JobCard__jobtitle___3HqOw truncate-lines truncate-lines-4']"))).get_attribute("onClick")
                # print(f"url: {url}")

                card_element = wait.until(EC.element_to_be_clickable((By.ID, f"job-card-{card_id}")))
                card = card_element.text
                # print(card)
                row_no += 1
                store_card_info(card, row_no)
                # """
                card_element.click()
                # sleep(uniform(1, 3))
                try:
                    job_description = wait.until(EC.presence_of_element_located((By.ID, "job_description"))).text
                    jobs.at[row_no, 'job_description'] = job_description
                    # print(job_description)
                except:
                    print("no job description")

                try:
                    posting_info = wait.until(EC.presence_of_element_located((By.XPATH, "//section[@id='job-details']/div[@class='w-70-l w-60-ms w-100 pr2-l pr2-ms relative']/div[@class='bg-white pa4'][1]/div[@class='jobInfo w-100 dib v-top relative']/section[2]"))).text
                    # print(posting_info)
                    applications, posted_date, closing_date = parse_posting_info(posting_info)

                    cur_date = datetime.strptime(posted_date, '%d %b %Y')

                    jobs.at[row_no, 'num_applications'] = applications
                    jobs.at[row_no, 'posted_date'] = posted_date
                    jobs.at[row_no, 'closing_date'] = closing_date
                    # print(f"{posted_date}, {closing_date}, {applications}")
                except:
                    print("no posted info")

                try: # <p data-testid="job-details-info-min-experience" class="black-80 f6 fw4 mv1 dib pr3 mr1 icon-bw-period">2 years exp</p>
                    experience = wait.until(EC.presence_of_element_located((By.XPATH, "//p[@data-testid='job-details-info-min-experience']"))).text.replace(" years exp", "").replace(" year exp", "")
                    jobs.at[row_no, 'experience'] = experience
                    # print(experience)
                except:
                    print("no experience")

                try:            
                    address = wait.until(EC.presence_of_element_located((By.XPATH, "//div[@class='db mt3 mb4']/section[@class='w-100 pb3 pb0-ns lh-copy']/p[@class='black-80 f6 fw4 mv1 dib pr3 mr1 icon-bw-location']/a[@class='link brand-sec underline-hover']"))).text
                    jobs.at[row_no, 'address'] = address
                    # print(address)
                except:
                    print("no address")
                
                driver.back()
                # """
            except TimeoutException as err:
                print(f"reached card no: {card_id}, error: {err.msg}")
                break            
                
        try:
            print('...next page')
            wait.until(EC.element_to_be_clickable((By.XPATH, "//section[@id='search-results']/div[@class='tc pv3']/button[@class='f5-5 pv2 ph3 mh1 dib black-80 hover-bg-white pointer'][3]"))).click()
            page_no += 1
        except TimeoutException as err:
            '''no more pages left'''
            proceed_to_next_page = False
    print("no more jobs to scrape")
    driver.quit()
    

In [13]:
# driver = webdriver.Chrome()
# wait = WebDriverWait(driver, 3)
# search_term = "hrwoerowhefo" 
# page_no = 0
# driver.get(f"https://www.mycareersfuture.gov.sg/search?search={search_term}&sortBy=new_posting_date&page={page_no}")
# try:
#     wait.until(EC.element_to_be_clickable((By.XPATH, "//section[@id='search-results']/div[@class='tc pv3']/button[@class='f5-5 pv2 ph3 mh1 dib black-80 hover-bg-white pointer'][3]"))).click()
#     sleep(5)
# except TimeoutException as err:
#     print(err.msg)
# driver.quit()

# Execute scraping

In [52]:
jobs = pd.DataFrame(columns=columns)

## AI

In [50]:
scrape_mycareersfuture("ai")

ROW NO: -1, CARD NO: 0, PAGE NO: 0
ROW NO: 0, CARD NO: 1, PAGE NO: 0
no posted info
ROW NO: 1, CARD NO: 2, PAGE NO: 0
ROW NO: 2, CARD NO: 3, PAGE NO: 0
ROW NO: 3, CARD NO: 4, PAGE NO: 0
ROW NO: 4, CARD NO: 5, PAGE NO: 0
no address
ROW NO: 5, CARD NO: 6, PAGE NO: 0
ROW NO: 6, CARD NO: 7, PAGE NO: 0
no experience
ROW NO: 7, CARD NO: 8, PAGE NO: 0
no experience
ROW NO: 8, CARD NO: 9, PAGE NO: 0
ROW NO: 9, CARD NO: 10, PAGE NO: 0
no experience
no address
ROW NO: 10, CARD NO: 11, PAGE NO: 0
no posted info
no experience
no address
ROW NO: 11, CARD NO: 12, PAGE NO: 0
no address
ROW NO: 12, CARD NO: 13, PAGE NO: 0
ROW NO: 13, CARD NO: 14, PAGE NO: 0
no address
ROW NO: 14, CARD NO: 15, PAGE NO: 0
ROW NO: 15, CARD NO: 16, PAGE NO: 0
no address
ROW NO: 16, CARD NO: 17, PAGE NO: 0
no address
ROW NO: 17, CARD NO: 18, PAGE NO: 0
no experience
ROW NO: 18, CARD NO: 19, PAGE NO: 0
no address
ROW NO: 19, CARD NO: 20, PAGE NO: 0
ROW NO: 20, CARD NO: 21, PAGE NO: 0
...next page
ROW NO: 21, CARD NO: 0, PAG

In [51]:
jobs

,company_name,recruiter_name,job_title,location,employment_type,job_level,job_function,salary_lower,salary_upper,salary_period,num_applications,posted_date,address,experience,closing_date,job_description
0,SINGAPORE-MIT ALLIANCE FOR RESEARCH AND TECHNO...,None,Postdoctoral Associate (Designing human-AI tea...,South,Contract,Professional,Engineering,8100,14000,Monthly,10,02 Apr 2024,1 Create Way 138602,1,02 May 2024,Roles & Responsibilities\nProject Overview\nWe...
1,SINGAPORE-MIT ALLIANCE FOR RESEARCH AND TECHNO...,None,Research Engineer (Designing human-AI teams) (CS),South,Contract,Professional,Engineering,5900,10000,Monthly,None,None,1 Create Way 138602,1,None,Roles & Responsibilities\nProject Overview\nWe...
2,SNOWFLAKE COMPUTING SINGAPORE PTE. LTD.,Recommended based on your skills & job applica...,Solutions Architect - AI / ML,Central,Permanent,Professional,Information Technology10 Years Exp,12000,24000,Monthly,25,18 Mar 2024,"SGX CENTRE II, 4 SHENTON WAY 068807",10,17 Apr 2024,Roles & Responsibilities\nSUMMARY\nWe are look...
3,SINGAPORE-MIT ALLIANCE FOR RESEARCH AND TECHNO...,None,Research Scientist (Designing human-AI teams) ...,South,Contract,Professional,Engineering,8900,17000,Monthly,17,02 Apr 2024,1 Create Way 138602,1,02 May 2024,Roles & Responsibilities\nProject Overview\nWe...
4,SINGAPORE-MIT ALLIANCE FOR RESEARCH AND TECHNO...,None,Postdoctoral Associate (Designing human-AI tea...,South,Contract,Professional,Sciences / Laboratory / R&D,8100,14000,Monthly,7,02 Apr 2024,1 Create Way 138602,1,02 May 2024,Roles & Responsibilities\nProject Overview\nWe...
5,SINGAPORE-MIT ALLIANCE FOR RESEARCH AND TECHNO...,None,Research Engineer (Designing human-AI teams) (SS),South,Contract,Professional,Sciences / Laboratory / R&D,5900,10000,Monthly,6,02 Apr 2024,None,1,02 May 2024,Roles & Responsibilities\nProject Overview\nWe...
6,SINGAPORE-MIT ALLIANCE FOR RESEARCH AND TECHNO...,None,Research Scientist (Designing human-AI teams) ...,South,Contract,Professional,Sciences / Laboratory / R&D,8900,17000,Monthly,14,02 Apr 2024,1 Create Way 138602,1,02 May 2024,Roles & Responsibilities\nProject Overview\nWe...
7,SINGAPORE-MIT ALLIANCE FOR RESEARCH AND TECHNO...,None,"Postdoctoral Associate (Ethical, accountable, ...",South,Contract,Professional,Engineering,8100,14000,Monthly,6,02 Apr 2024,1 Create Way 138602,None,02 May 2024,Roles & Responsibilities\nProject Overview\nWe...
8,SINGAPORE-MIT ALLIANCE FOR RESEARCH AND TECHNO...,None,Postdoctoral Associate (AI for Human Capital D...,South,Contract,Professional,Engineering,8100,14000,Monthly,5,02 Apr 2024,1 Create Way 138602,None,02 May 2024,Roles & Responsibilities\nProject Overview\nWe...
9,LAWNET TECHNOLOGY SERVICES PTE. LTD.,None,AI Engineer,East,Full Time,Middle Management,Information Technology,5300,8700,Monthly,2,02 Apr 2024,"THE ADELPHI, 1 COLEMAN STREET 179803",7,02 May 2024,Roles & Responsibilities\nAbout the role\nAs a...


In [60]:
jobs.to_csv("jobs_ai.csv", encoding='utf-8', index=False)
jobs = pd.DataFrame(columns=columns)

## Data analytics

In [25]:
scrape_mycareersfuture("data%20analytics")

ROW NO: -1, CARD NO: 0, PAGE NO: 0
ROW NO: 0, CARD NO: 1, PAGE NO: 0
ROW NO: 1, CARD NO: 2, PAGE NO: 0
no address
ROW NO: 2, CARD NO: 3, PAGE NO: 0
ROW NO: 3, CARD NO: 4, PAGE NO: 0
ROW NO: 4, CARD NO: 5, PAGE NO: 0
ROW NO: 5, CARD NO: 6, PAGE NO: 0
ROW NO: 6, CARD NO: 7, PAGE NO: 0
no address
ROW NO: 7, CARD NO: 8, PAGE NO: 0
ROW NO: 8, CARD NO: 9, PAGE NO: 0
no job description
no address
ROW NO: 9, CARD NO: 10, PAGE NO: 0
ROW NO: 10, CARD NO: 11, PAGE NO: 0
ROW NO: 11, CARD NO: 12, PAGE NO: 0
no experience
ROW NO: 12, CARD NO: 13, PAGE NO: 0
no experience
no address
ROW NO: 13, CARD NO: 14, PAGE NO: 0
ROW NO: 14, CARD NO: 15, PAGE NO: 0
no address
ROW NO: 15, CARD NO: 16, PAGE NO: 0
no address
ROW NO: 16, CARD NO: 17, PAGE NO: 0
no address
ROW NO: 17, CARD NO: 18, PAGE NO: 0
no address
ROW NO: 18, CARD NO: 19, PAGE NO: 0
no address
ROW NO: 19, CARD NO: 20, PAGE NO: 0
reached card no: 20, error: 
...next page
ROW NO: 19, CARD NO: 0, PAGE NO: 1
ROW NO: 20, CARD NO: 1, PAGE NO: 1
ROW NO

In [26]:
jobs

,company_name,recruiter_name,job_title,location,employment_type,job_level,job_function,salary_lower,salary_upper,salary_period,num_applications,posted_date,address,experience,closing_date,job_description
0,CPG CORPORATION PTE LTD,CPG CONSULTANTS PTE LTD,Senior / Engineer (Applications and Data Analy...,West,Contract,Executive,Architecture / Interior Design,1.0,NaN,Posted today,1,02 Apr 2024,"WESTGATE TOWER, 1 GATEWAY DRIVE 608531",3,02 May 2024,Roles & Responsibilities\nJob Responsibilities...
1,TENTEN PARTNERS PTE. LTD.,None,Financial Crime Compliance Analytics Manager,Central,Permanent,Executive,Banking And Finance,1.0,NaN,Posted today,1,02 Apr 2024,"MYP CENTRE, 9 BATTERY ROAD 049910",3,02 May 2024,Roles & Responsibilities\nJob Description:\n\n...
2,COMPIT CONSULTANTS PTE LTD,None,Data Engineer,Islandwide,Contract,Professional,Consulting,0.0,NaN,Posted today,0,02 Apr 2024,None,5,02 May 2024,Roles & Responsibilities\nWe are looking Data ...
3,ADECCO PERSONNEL PTE LTD,None,Data Analytics,Central,Contract,Executive,Junior Others,4.0,NaN,Posted today,4,02 Apr 2024,"SHAW CENTRE, 1 SCOTTS ROAD 228208",2,16 Apr 2024,Roles & Responsibilities\nDevelop and improve ...
4,EPS CONSULTANTS PTE LTD,None,Research and Data Maintenance Executive (6 Mon...,Central,Contract,Executive,Admin / Secretarial,0.0,NaN,Posted today,0,02 Apr 2024,160 ROBINSON ROAD 068914,1,02 May 2024,"Roles & Responsibilities\n6-Month contract, hi..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59,IDC TECHNOLOGIES (SINGAPORE) PTE. LTD.,None,Data Engineer,Central,Contract,Professional,Information Technology,13.0,NaN,Posted 11 days ago,13,22 Mar 2024,"GB BUILDING, 143 CECIL STREET 069542",None,05 Apr 2024,Roles & Responsibilities\nResponsibilities:\nD...
60,SAP ASIA PTE. LTD.,None,Associate AI Scientist / AI Scientist – Artifi...,South,Permanent,Executive,Information Technology,6000.0,9000.0,Monthly,None,None,None,None,None,None
61,TIKTOK PTE. LTD.,None,Data Scientist-User Growth Tiktok,Central,Full Time,Executive,Engineering,10000.0,20000.0,Monthly,None,None,None,None,None,None
62,NATIONAL UNIVERSITY OF SINGAPORE,None,Research Fellow (IDS-Cisco),South,Full Time,Professional,Sciences / Laboratory / R&D,6000.0,9000.0,Monthly,None,None,None,None,None,None


In [27]:
jobs.to_csv("jobs_data-analytics.csv", encoding='utf-8', index=False)
jobs = pd.DataFrame(columns=columns)

## Data science

In [53]:
scrape_mycareersfuture("data%20science")

ROW NO: -1, CARD NO: 0, PAGE NO: 0
ROW NO: 0, CARD NO: 1, PAGE NO: 0
ROW NO: 1, CARD NO: 2, PAGE NO: 0
ROW NO: 2, CARD NO: 3, PAGE NO: 0
no address
ROW NO: 3, CARD NO: 4, PAGE NO: 0
no address
ROW NO: 4, CARD NO: 5, PAGE NO: 0
ROW NO: 5, CARD NO: 6, PAGE NO: 0
ROW NO: 6, CARD NO: 7, PAGE NO: 0
no posted info
ROW NO: 7, CARD NO: 8, PAGE NO: 0
no address
ROW NO: 8, CARD NO: 9, PAGE NO: 0
no address
ROW NO: 9, CARD NO: 10, PAGE NO: 0
ROW NO: 10, CARD NO: 11, PAGE NO: 0
no experience
no address
ROW NO: 11, CARD NO: 12, PAGE NO: 0
no experience
no address
ROW NO: 12, CARD NO: 13, PAGE NO: 0
ROW NO: 13, CARD NO: 14, PAGE NO: 0
ROW NO: 14, CARD NO: 15, PAGE NO: 0
no address
ROW NO: 15, CARD NO: 16, PAGE NO: 0
ROW NO: 16, CARD NO: 17, PAGE NO: 0
ROW NO: 17, CARD NO: 18, PAGE NO: 0
ROW NO: 18, CARD NO: 19, PAGE NO: 0
ROW NO: 19, CARD NO: 20, PAGE NO: 0
no experience
ROW NO: 20, CARD NO: 21, PAGE NO: 0
...next page
ROW NO: 21, CARD NO: 0, PAGE NO: 1
no posted info
no address
ROW NO: 22, CARD NO:

In [54]:
jobs

,company_name,recruiter_name,job_title,location,employment_type,job_level,job_function,salary_lower,salary_upper,salary_period,num_applications,posted_date,address,experience,closing_date,job_description
0,OKRATIN SOLUTIONS PTE. LTD.,None,Clinical Nurse,Central,Full Time,Executive,Medical / Therapy Services,8000,15000,Monthly,0,02 Apr 2024,"SGX CENTRE I, 2 SHENTON WAY 068804",5,16 Apr 2024,Roles & Responsibilities\nAbout:\nOkratin is a...
1,OKRATIN SOLUTIONS PTE. LTD.,None,Sales Manager,Central,Full Time,Senior Executive,Sales / Retail,5000,15000,Monthly,4,02 Apr 2024,"SGX CENTRE I, 2 SHENTON WAY 068804",5,02 May 2024,Roles & Responsibilities\nAbout:\nOkratin is a...
2,ST ENGINEERING URBAN SOLUTIONS LTD.,Recommended based on your skills & job applica...,"Assistant Principal Engineer, Artificial Intel...",West,Full Time,Manager,Engineering3 Years Exp,4000,8000,Monthly,6,07 Mar 2024,100 JURONG EAST STREET 21 609602,3,06 Apr 2024,Roles & Responsibilities\nJob Description:\nMa...
3,CONSTRUCTOR TECHNOLOGY PTE. LTD.,None,HR Generalist,Central,Full Time,Executive,Human Resources,4000,7000,Monthly,4,02 Apr 2024,None,5,23 Apr 2024,Roles & Responsibilities\nAbout Constructor\n\...
4,QUESS SELECTION & SERVICES PTE. LTD.,None,"Data Developer (Python, Kubernetes, API)",Islandwide,Contract,Professional,Information Technology,6500,8000,Monthly,1,02 Apr 2024,None,3,16 Apr 2024,Roles & Responsibilities\nRoles & Responsibili...
5,INFOSYS LIMITED SINGAPORE BRANCH,None,Test Manager,South,Permanent,Manager,Information Technology,8000,12500,Monthly,4,02 Apr 2024,"KEPPEL BAY TOWER, 1 HARBOURFRONT AVENUE 098632",10,02 May 2024,Roles & Responsibilities\nAbout Infosys:\nInfo...
6,NTT DATA SINGAPORE PTE. LTD.,None,Data Scientist ( 9 months contract ),Central,Contract,Professional,Information Technology,6000,8000,Monthly,3,02 Apr 2024,"REPUBLIC PLAZA, 9 RAFFLES PLACE 048619",4,02 May 2024,Roles & Responsibilities\nData Scientist Healt...
7,SINGAPORE-MIT ALLIANCE FOR RESEARCH AND TECHNO...,None,Postdoctoral Associate (Data-efficient machine...,South,Contract,Professional,Sciences / Laboratory / R&D,8100,14000,Monthly,None,None,1 Create Way 138602,1,None,Roles & Responsibilities\nJob Description\nWe ...
8,REMOTE TECH SINGAPORE PTE. LTD.,None,"Senior Product Manager, Economic Design",Islandwide,Full Time,Professional,Consulting,15000,18000,Monthly,5,02 Apr 2024,None,3,16 Apr 2024,Roles & Responsibilities\nJob Overview:\nYou w...
9,SPRAOI SOFTWARE SERVICES PTE. LTD.,None,Teradata Engineer (Contract),Central,Contract,Professional,Information Technology,9000,12500,Monthly,1,02 Apr 2024,None,8,16 Apr 2024,Roles & Responsibilities\nRole:\n· Provide tec...


In [55]:
jobs.to_csv("jobs_data-science.csv", encoding='utf-8', index=False)
jobs = pd.DataFrame(columns=columns)

## Machine learning

In [56]:
scrape_mycareersfuture("machine%20learning")

ROW NO: -1, CARD NO: 0, PAGE NO: 0
ROW NO: 0, CARD NO: 1, PAGE NO: 0
no address
ROW NO: 1, CARD NO: 2, PAGE NO: 0
ROW NO: 2, CARD NO: 3, PAGE NO: 0
no address
ROW NO: 3, CARD NO: 4, PAGE NO: 0
ROW NO: 4, CARD NO: 5, PAGE NO: 0
no experience
ROW NO: 5, CARD NO: 6, PAGE NO: 0
ROW NO: 6, CARD NO: 7, PAGE NO: 0
no experience
ROW NO: 7, CARD NO: 8, PAGE NO: 0
ROW NO: 8, CARD NO: 9, PAGE NO: 0
no experience
ROW NO: 9, CARD NO: 10, PAGE NO: 0
ROW NO: 10, CARD NO: 11, PAGE NO: 0
ROW NO: 11, CARD NO: 12, PAGE NO: 0
ROW NO: 12, CARD NO: 13, PAGE NO: 0
no address
ROW NO: 13, CARD NO: 14, PAGE NO: 0
ROW NO: 14, CARD NO: 15, PAGE NO: 0
no posted info
no experience
ROW NO: 15, CARD NO: 16, PAGE NO: 0
ROW NO: 16, CARD NO: 17, PAGE NO: 0
ROW NO: 17, CARD NO: 18, PAGE NO: 0
ROW NO: 18, CARD NO: 19, PAGE NO: 0
ROW NO: 19, CARD NO: 20, PAGE NO: 0
ROW NO: 20, CARD NO: 21, PAGE NO: 0
no address
...next page
ROW NO: 21, CARD NO: 0, PAGE NO: 1
no address
ROW NO: 22, CARD NO: 1, PAGE NO: 1
ROW NO: 23, CARD NO

In [57]:
jobs

,company_name,recruiter_name,job_title,location,employment_type,job_level,job_function,salary_lower,salary_upper,salary_period,num_applications,posted_date,address,experience,closing_date,job_description
0,LAWNET TECHNOLOGY SERVICES PTE. LTD.,None,NLP / Data Engineer,East,Full Time,Professional,Information Technology,5300,8700,Monthly,0,02 Apr 2024,"THE ADELPHI, 1 COLEMAN STREET 179803",7,02 May 2024,Roles & Responsibilities\nAbout the role:\nAs ...
1,QUESS SELECTION & SERVICES PTE. LTD.,None,"Data Developer (Python, Kubernetes, API)",Islandwide,Contract,Professional,Information Technology,6500,8000,Monthly,1,02 Apr 2024,None,3,16 Apr 2024,Roles & Responsibilities\nRoles & Responsibili...
2,AIRWALLEX (SINGAPORE) PTE. LTD.,Recommended based on your skills & job applica...,"Senior Data Scientist, Algorithm",Central,Full Time,Senior Executive,Engineering5 Years Exp,11250,22500,Monthly,61,14 Mar 2024,"PRUDENTIAL TOWER, 30 CECIL STREET 049712",5,13 Apr 2024,Roles & Responsibilities\nResponsibilities\nDe...
3,ENCORA TECHNOLOGIES PTE. LTD.,None,Technical Cloud Solution Architect,Islandwide,Contract,Middle Management,Information Technology,7000,12500,Monthly,0,02 Apr 2024,None,6,16 Apr 2024,Roles & Responsibilities\nRequirements\n3+ yea...
4,EXECUTIVE SEARCH PRIVATE LIMITED,None,R&D Engineer (FPGA / Linux) - Electronics Indu...,East,Permanent,Senior Executive,Design,4000,6000,Monthly,1,02 Apr 2024,15 Changi Business Park 486057,3,09 Apr 2024,Roles & Responsibilities\nROLE AND RESPONSIBIL...
5,NANYANG TECHNOLOGICAL UNIVERSITY,None,Research Fellow (Computer Science / Machine Le...,West,Contract,Professional,Sciences / Laboratory / R&D,6696,11603,Monthly,0,02 Apr 2024,"NANYANG TECHNOLOGICAL UNIVERSITY, 50 NANYANG A...",None,16 Apr 2024,Roles & Responsibilities\nThe S-Lab for Advanc...
6,KLA-TENCOR (SINGAPORE) PTE. LTD.,None,Algorithm Engineer (R&D),North,Permanent,Professional,Engineering,6000,11000,Monthly,1,02 Apr 2024,23 SERANGOON NORTH AVENUE 5 554530,5,02 May 2024,Roles & Responsibilities\nResponsibilities:\nT...
7,NANYANG TECHNOLOGICAL UNIVERSITY,None,Project Officer (Computer Engineering / Comput...,West,Contract,Professional,Sciences / Laboratory / R&D,4522,7533,Monthly,0,02 Apr 2024,"NANYANG TECHNOLOGICAL UNIVERSITY, 50 NANYANG A...",None,16 Apr 2024,Roles & Responsibilities\nS-Lab for Advanced I...
8,NTT DATA SINGAPORE PTE. LTD.,None,Data Scientist ( 9 months contract ),Central,Contract,Professional,Information Technology,6000,8000,Monthly,3,02 Apr 2024,"REPUBLIC PLAZA, 9 RAFFLES PLACE 048619",4,02 May 2024,Roles & Responsibilities\nData Scientist Healt...
9,SINGAPORE-MIT ALLIANCE FOR RESEARCH AND TECHNO...,None,Postdoctoral Associate (Human-Robot Interactio...,South,Contract,Professional,Sciences / Laboratory / R&D,8100,14000,Monthly,3,02 Apr 2024,1 Create Way 138602,None,02 May 2024,Roles & Responsibilities\nProject Overview\nWe...


In [58]:
jobs.to_csv("jobs_ml.csv", encoding='utf-8', index=False)
jobs = pd.DataFrame(columns=columns)